# JetBot - Data collection without gamecontroller

In this notebook we'll collect training data for CNN VAE. The training data save to dataset directory.

## Import module



In [1]:
import os
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg


## Show log_button

If you enable log_button then start recording images.


In [2]:
log_button = widgets.ToggleButton(value=False, description='enable logging')
display(log_button)

Controller()

## Initialize Camera

Next is initializing camera module. Image size is 320 x 240. Frame rate is about 27Hz. We'll save image in camera observer method. camera observer method can get image per frame rate. Thus, frame rate is decide to image save interval.

In [6]:
camera = Camera.instance(width=320, height=240)
image = widgets.Image(format='jpeg', width=320, height=240)
camera_link = traitlets.dlink((camera,'value'), (image,'value'), transform=bgr8_to_jpeg)


## UI Widget


In [7]:
DATASET_DIR = 'dataset'
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

dataset=DATASET_DIR
layout = widgets.Layout(width='100px', height='64px')
count_box   = widgets.IntText(layout=layout, value=len(os.listdir(dataset)))
count_label = widgets.Label(layout=layout, value='Number image:')
count_panel = widgets.HBox([count_label,count_box])

panel = widgets.VBox([count_panel])
display(widgets.HBox([panel,image]))


Directories not created becasue they already exist


## Set callback for collect the training data.

```save_record``` is callback for training data. The method set to camera observer. This callback saving the image to DATASET_DIR. When click ```enable logging``` button, this method recording training data. You can check number of training data with ```Number image text box```.

In [8]:

import os
from uuid import uuid1

def save_record(change):                
    if log_button.value:
        
        image_name = '{}.jpg'.format(uuid1())
        image_path = os.path.join(DATASET_DIR, image_name)
        save_image=bgr8_to_jpeg(change['new'])
        with open(image_path, 'wb') as f:
            f.write(save_image)
        count_box.value = len(os.listdir(dataset)) 


save_record({'new': camera.value})
camera.observe(save_record, names='value')

## Cleanup

After collecting enough data. cleanup camera observer and stop all motor.

In [9]:
camera.unobserve(save_record, names='value')
camera_link.unlink()


## Cleate dataset.zip file 

In [10]:
import datetime
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q jetbot_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}